In [1]:
!pip install ultralytics opencv-python

  Using cached ultralytics-8.3.152-py3-none-any.whl.metadata (37 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.152-py3-none-any.whl (1.0 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os
import json
import logging
from pathlib import Path
from ultralytics import YOLO
import cv2
import numpy as np

# Ensure logs and output directories exist
os.makedirs('logs', exist_ok=True)
os.makedirs('data/output', exist_ok=True)

# Configure logging
logging.basicConfig(
    filename='logs/app.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def preprocess_frame(frame):
    """
    Preprocess frame for better detection (resize and normalize).
    
    Args:
        frame (np.ndarray): Input frame.
    
    Returns:
        np.ndarray: Preprocessed frame.
    """
    # Resize to 640x640 (YOLOv8 default input size)
    frame = cv2.resize(frame, (640, 640), interpolation=cv2.INTER_AREA)
    # Normalize pixel values to [0, 1]
    frame = frame / 255.0
    return frame

def detect_objects(frame_path, model, confidence_threshold=0.3):
    """
    Detect fashion items in a frame using YOLOv8.
    
    Args:
        frame_path (str): Path to the frame image.
        model (YOLO): Pretrained YOLOv8 model.
        confidence_threshold (float): Minimum confidence score for detections.
    
    Returns:
        list: List of detections with class, bounding box, confidence, and frame info.
    """
    try:
        # Read and preprocess frame
        frame = cv2.imread(frame_path)
        if frame is None:
            logging.error(f"Failed to read frame: {frame_path}")
            return []
        
        frame = preprocess_frame(frame)
        
        # Perform detection
        results = model(frame, conf=confidence_threshold)
        
        detections = []
        all_detections = []  # For debugging
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls)
                class_name = model.names[class_id]
                confidence = float(box.conf)
                x, y, w, h = map(int, box.xywh[0])  # Bounding box (center x, y, width, height)
                
                # Store all detections for debugging
                all_detections.append({
                    'class_name': class_name,
                    'confidence': confidence,
                    'bounding_box': {'x': x, 'y': y, 'w': w, 'h': h}
                })
                
                # Filter for fashion-related classes or person (fallback)
                if class_name in ['top', 'bottom', 'dress', 'jacket', 'earrings', 'bag', 'shoes', 'person']:
                    detections.append({
                        'class_name': class_name,
                        'bounding_box': {'x': x, 'y': y, 'w': w, 'h': h},
                        'confidence': confidence,
                        'frame_path': frame_path
                    })
        
        # Log all detections for debugging
        logging.info(f"All detections in {frame_path}: {json.dumps(all_detections, indent=2)}")
        logging.info(f"Filtered {len(detections)} objects in {frame_path}")
        return detections
    
    except Exception as e:
        logging.error(f"Error detecting objects in {frame_path}: {str(e)}")
        return []

def process_video_frames(video_id, frames_dir, output_dir, model):
    """
    Process all frames for a video and save detection results.
    
    Args:
        video_id (str): ID of the video (e.g., 'abc123').
        frames_dir (str): Directory containing frames for the video.
        output_dir (str): Directory to save detection JSON.
        model (YOLO): Pretrained YOLOv8 model.
    
    Returns:
        list: All detections for the video.
    """
    try:
        all_detections = []
        frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.jpg')])
        
        for frame_file in frame_files:
            frame_path = os.path.join(frames_dir, frame_file)
            frame_number = int(frame_file.split('_')[1].split('.')[0])  # e.g., frame_0001.jpg -> 1
            
            # Detect objects in frame
            detections = detect_objects(frame_path, model)
            for detection in detections:
                detection['frame_number'] = frame_number
                all_detections.append(detection)
        
        # Save detections to JSON
        output_path = os.path.join(output_dir, f"{video_id}_detections.json")
        with open(output_path, 'w') as f:
            json.dump({
                'video_id': video_id,
                'detections': all_detections
            }, f, indent=2)
        
        logging.info(f"Saved {len(all_detections)} detections for video {video_id} to {output_path}")
        return all_detections
    
    except Exception as e:
        logging.error(f"Error processing video {video_id}: {str(e)}")
        return []

def main():
    # Directories
    frames_base_dir = "frames"
    output_dir = "data/output"
    model_path = "models/yolo/yolov8m.pt"  # Pretrained YOLOv8 model
    
    # Load YOLOv8 model
    try:
        model = YOLO(model_path)
        logging.info(f"Loaded YOLOv8 model from {model_path}")
        logging.info(f"Model classes: {model.names}")
    except Exception as e:
        logging.error(f"Failed to load YOLOv8 model: {str(e)}")
        return
    
    # Process each video's frames
    for video_id in os.listdir(frames_base_dir):
        frames_dir = os.path.join(frames_base_dir, video_id)
        if os.path.isdir(frames_dir):
            logging.info(f"Processing frames for video: {video_id}")
            process_video_frames(video_id, frames_dir, output_dir, model)

if __name__ == "__main__":
    main()


0: 640x640 (no detections), 1187.5ms
Speed: 25.9ms preprocess, 1187.5ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1108.6ms
Speed: 27.9ms preprocess, 1108.6ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1174.8ms
Speed: 24.2ms preprocess, 1174.8ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1322.3ms
Speed: 27.8ms preprocess, 1322.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1069.7ms
Speed: 23.1ms preprocess, 1069.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1073.5ms
Speed: 22.6ms preprocess, 1073.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1114.2ms
Speed: 24.6ms preprocess, 1114.2ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1271.

In [7]:
from ultralyticsplus import YOLO, render_result

# load model
model = YOLO('kesimeg/yolov8n-clothing-detection')

# set image
image = './frames/2025-05-28_13-42-32_UTC/frame_0540.jpg"'

# perform inference
results = model.predict(image)

# observe results
print(results[0].boxes)
render = render_result(model=model, image=image, result=results[0])
render.show()


ModuleNotFoundError: No module named 'ultralyticsplus'

In [6]:
import os
import json
import logging
from ultralytics import YOLO

# Ensure logs and output directories exist
os.makedirs('logs', exist_ok=True)
os.makedirs('data/output', exist_ok=True)

# Configure logging
logging.basicConfig(
    filename='logs/app.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def check_model_classes(model_path):
    """
    Load a YOLOv8 model and print its class names, highlighting clothing-related classes.
    
    Args:
        model_path (str): Path to the YOLOv8 model file (e.g., yolov8m.pt).
    
    Returns:
        dict: Dictionary of class ID to class name.
    """
    try:
        # Load YOLOv8 model
        model = YOLO(model_path)
        class_names = model.names
        logging.info(f"Loaded YOLOv8 model from {model_path}")
        logging.info(f"Total classes: {len(class_names)}")
        
        # Define clothing-related classes to highlight
        clothing_related = ['top', 'bottom', 'dress', 'jacket', 'earrings', 'bag', 'shoes', 
                          'handbag', 'tie', 'suit', 'shirt', 'person']
        
        # Print and log all class names
        print("YOLOv8 Model Class Names:")
        clothing_classes = []
        for class_id, class_name in class_names.items():
            is_clothing = class_name in clothing_related
            status = " (Clothing-related)" if is_clothing else ""
            print(f"ID {class_id}: {class_name}{status}")
            logging.info(f"Class ID {class_id}: {class_name}{status}")
            if is_clothing:
                clothing_classes.append(class_name)
        
        # Log clothing-related classes specifically
        logging.info(f"Clothing-related classes found: {clothing_classes}")
        
        # Save class names to JSON for reference
        output_path = "data/output/yolo_class_names.json"
        with open(output_path, 'w') as f:
            json.dump({
                'model_path': model_path,
                'class_names': class_names,
                'clothing_classes': clothing_classes
            }, f, indent=2)
        logging.info(f"Saved class names to {output_path}")
        
        return class_names
    
    except Exception as e:
        logging.error(f"Error loading model {model_path}: {str(e)}")
        print(f"Error: {str(e)}")
        return {}

def main():
    model_path = "models/yolo/yolov8m.pt"  # Path to pretrained YOLOv8 model
    check_model_classes(model_path)

if __name__ == "__main__":
    main()

YOLOv8 Model Class Names:
ID 0: person (Clothing-related)
ID 1: bicycle
ID 2: car
ID 3: motorcycle
ID 4: airplane
ID 5: bus
ID 6: train
ID 7: truck
ID 8: boat
ID 9: traffic light
ID 10: fire hydrant
ID 11: stop sign
ID 12: parking meter
ID 13: bench
ID 14: bird
ID 15: cat
ID 16: dog
ID 17: horse
ID 18: sheep
ID 19: cow
ID 20: elephant
ID 21: bear
ID 22: zebra
ID 23: giraffe
ID 24: backpack
ID 25: umbrella
ID 26: handbag (Clothing-related)
ID 27: tie (Clothing-related)
ID 28: suitcase
ID 29: frisbee
ID 30: skis
ID 31: snowboard
ID 32: sports ball
ID 33: kite
ID 34: baseball bat
ID 35: baseball glove
ID 36: skateboard
ID 37: surfboard
ID 38: tennis racket
ID 39: bottle
ID 40: wine glass
ID 41: cup
ID 42: fork
ID 43: knife
ID 44: spoon
ID 45: bowl
ID 46: banana
ID 47: apple
ID 48: sandwich
ID 49: orange
ID 50: broccoli
ID 51: carrot
ID 52: hot dog
ID 53: pizza
ID 54: donut
ID 55: cake
ID 56: chair
ID 57: couch
ID 58: potted plant
ID 59: bed
ID 60: dining table
ID 61: toilet
ID 62: tv
ID 6